In [4]:
import numpy as np
import os
from astropy.io import fits
import matplotlib.pyplot as plt

Region = "ngc5068"
Band = "F770W"
synthetic_dir = r"C:\Users\jhoffm72\Documents\FilPHANGS\Data\ngc5068_F770W\SyntheticMap"

original_image_path = r"C:\Users\jhoffm72\Documents\FilPHANGS\Data\OriginalImages\ngc5068_F770W_JWST_Emission_starsub.fits"
with fits.open(original_image_path) as hdul:
    orig_image_data = hdul[0].data
    header = hdul[0].header

orig_image_data = np.nan_to_num(orig_image_data, nan=0.0)  # Replace NaNs with 0


reconstructed_image = np.zeros_like(orig_image_data)

for file in os.listdir(synthetic_dir):
    file_path = os.path.join(synthetic_dir, file)
    if not file_path.endswith('.fits') and "Reconstruction" in file_path or "Subtraction" in file_path:  # Skip non-FITS files
        continue
    print(file_path)
    try:
        with fits.open(file_path) as hdul:
            image_data = hdul[0].data
        if np.shape(image_data) != np.shape(orig_image_data):
            print(f"Shape mismatch: Original image shape {np.shape(orig_image_data)}, synthetic map {file} shape: {np.shape(image_data)}")
            continue  # Skip mismatched files
        reconstructed_image += image_data
        print(np.max(reconstructed_image))


        save_path = os.path.join(synthetic_dir, f"Reconstruction_of_Original_{Region}_{Band}.fits")
        hdu = fits.PrimaryHDU(reconstructed_image, header=header)
        hdu.writeto(save_path, overwrite=True)

        subtracted_image = orig_image_data - reconstructed_image
        save_path = os.path.join(synthetic_dir, f"SubtrcationOfOriginalAndSynthetic_{Region}_{Band}.fits")
        hdu = fits.PrimaryHDU(subtracted_image, header=header)
        hdu.writeto(save_path, overwrite=True)

    except:
        print('skip')


C:\Users\jhoffm72\Documents\FilPHANGS\Data\ngc5068_F770W\SyntheticMap\ngc5068_F770W_JWST_Emission_starsub_CDDss0008pc.fits_DensityData.csv
skip
C:\Users\jhoffm72\Documents\FilPHANGS\Data\ngc5068_F770W\SyntheticMap\ngc5068_F770W_JWST_Emission_starsub_CDDss0008pc.fits_SyntheticMap.fits
69.7844568490349
C:\Users\jhoffm72\Documents\FilPHANGS\Data\ngc5068_F770W\SyntheticMap\ngc5068_F770W_JWST_Emission_starsub_CDDss0016pc.fits_DensityData.csv
skip
C:\Users\jhoffm72\Documents\FilPHANGS\Data\ngc5068_F770W\SyntheticMap\ngc5068_F770W_JWST_Emission_starsub_CDDss0016pc.fits_SyntheticMap.fits
nan
C:\Users\jhoffm72\Documents\FilPHANGS\Data\ngc5068_F770W\SyntheticMap\ngc5068_F770W_JWST_Emission_starsub_CDDss0032pc.fits_DensityData.csv
skip
C:\Users\jhoffm72\Documents\FilPHANGS\Data\ngc5068_F770W\SyntheticMap\ngc5068_F770W_JWST_Emission_starsub_CDDss0032pc.fits_SyntheticMap.fits
nan
C:\Users\jhoffm72\Documents\FilPHANGS\Data\ngc5068_F770W\SyntheticMap\ngc5068_F770W_JWST_Emission_starsub_CDDss0064pc.fi